<a href="https://colab.research.google.com/github/pachterlab/CP_2023/blob/main/notebooks/celltypePred_Fig4a/kimetal_tenx_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import os


from tqdm import tnrange, tqdm_notebook
def download_file(doi,ext):
	url = 'https://api.datacite.org/dois/'+doi+'/media'
	r = requests.get(url).json()
	netcdf_url = r['data'][0]['attributes']['url']
	r = requests.get(netcdf_url,stream=True)
	#Set file name
	fname = doi.split('/')[-1]+ext
	#Download file with progress bar
	if r.status_code == 403:
		print("File Unavailable")
	if 'content-length' not in r.headers:
		print("Did not get file")
	else:
		with open(fname, 'wb') as f:
			total_length = int(r.headers.get('content-length'))
			pbar = tnrange(int(total_length/1024), unit="B")
			for chunk in r.iter_content(chunk_size=1024):
				if chunk:
					pbar.update()
					f.write(chunk)
		return fname


#10x VMH data
#metadata.csv
download_file('10.22002/D1.2065','.gz')

#tenx.mtx (log counts)
download_file('10.22002/D1.2072','.gz')

#10X raw Count Matrix
download_file('10.22002/D1.2073','.gz')

#var.csv
download_file('10.22002/D1.2066','.gz')



os.system("gunzip *.gz")

os.system("mv D1.2065 metadata.csv")
os.system("mv D1.2072 tenx.mtx")
os.system("mv D1.2073 tenxCount.mtx")
os.system("mv D1.2066 tenx_var.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`


  0%|          | 0/940 [00:00<?, ?B/s]

  0%|          | 0/104912 [00:00<?, ?B/s]

  0%|          | 0/14752 [00:00<?, ?B/s]

  0%|          | 0/12 [00:00<?, ?B/s]

0

In [ ]:
%cd /content

/content


In [ ]:
!git clone https://github.com/hhcho/densvis.git

Cloning into 'densvis'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 145 (delta 3), reused 2 (delta 2), pack-reused 137
Receiving objects: 100% (145/145), 419.61 KiB | 10.49 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [ ]:
%cd /content/densvis/densne/

/content/densvis/densne


In [ ]:
!g++ sptree.cpp densne.cpp densne_main.cpp -o den_sne -O2
import densne

densne.cpp: In function ‘bool DENSNE::load_data(double**, int*, int*, int*, double*, double*, int*, int*, double*, double*, bool*, double**)’:
densne.cpp:961:8: warning: ignoring return value of ‘size_t fread(void*, size_t, size_t, FILE*)’, declared with attribute warn_unused_result [-Wunused-result]
   fread(n, sizeof(int), 1, h);                                            // number of datapoints
   ~~~~~^~~~~~~~~~~~~~~~~~~~~~
densne.cpp:962:8: warning: ignoring return value of ‘size_t fread(void*, size_t, size_t, FILE*)’, declared with attribute warn_unused_result [-Wunused-result]
   fread(d, sizeof(int), 1, h);                                            // original dimensionality
   ~~~~~^~~~~~~~~~~~~~~~~~~~~~
densne.cpp:963:8: warning: ignoring return value of ‘size_t fread(void*, size_t, size_t, FILE*)’, declared with attribute warn_unused_result [-Wunused-result]
   fread(theta, sizeof(double), 1, h);                                        // gradient accuracy
   ~~~~~^~~~~~~~~~

In [ ]:
%cd /content/

/content


In [ ]:
!git clone https://github.com/pachterlab/CP_2023.git

Cloning into 'CP_2022'...
remote: Enumerating objects: 470, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 470 (delta 19), reused 19 (delta 5), pack-reused 424
Receiving objects: 100% (470/470), 159.66 MiB | 23.98 MiB/s, done.
Resolving deltas: 100% (232/232), done.


In [ ]:
!pip3 install --quiet torch
!pip3 install --quiet anndata
!pip3 install --quiet matplotlib
!pip3 install --quiet scikit-learn
!pip3 install --quiet torchsummary
!pip install --quiet scanpy

     |████████████████████████████████| 96 kB 2.6 MB/s 
     |████████████████████████████████| 7.7 MB 7.8 MB/s 
     |████████████████████████████████| 88 kB 6.7 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 1.1 MB 43.7 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 296 kB 9.7 MB/s 
     |████████████████████████████████| 145 kB 48.2 MB/s 
     |████████████████████████████████| 85 kB 3.8 MB/s 
     |████████████████████████████████| 722 kB 59.0 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 708 kB 51.5 MB/s 
     |████████████████████████████████| 529 kB 50.2 MB/s 
     |████████████████████████████████| 185 kB 62.1 MB/s 
     |████████████████████████████████| 292 kB 53.7 MB/s 
     |████████████████████████████████| 237 kB 60.3 MB/s 
     |████████████████████████████████| 51 kB 6.2 MB/s 
     |████████████████████████

In [ ]:
%cd /content/CP_2023/scripts

/content/CP_2022/scripts


## **Install Packages**

In [ ]:
import anndata
import pandas as pd
import numpy as np
from MCML import MCML #Now has continuous label addition
from Picasso import Picasso
import tools as tl
import random

from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.neighbors import NeighborhoodComponentsAnalysis, NearestNeighbors
from sklearn.metrics import pairwise_distances
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import scale
import torch
import time
import scanpy as sc
import seaborn as sns
import umap
from scipy import stats
import scipy.io as sio
sns.set_style('white')

INFO:pytorch_lightning.utilities.seed:Global seed set to 0
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/warnings.py:54: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  "pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6"
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


## **Import Data**

In [ ]:
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams['axes.linewidth'] = 0.1

state = 42
ndims = 2

data_path = '../..'

pcs = 50
pcs2 = 100
pcs3 = 40 #From study
n_latent = 50

In [ ]:
count_mat = sio.mmread(data_path+'/tenx.mtx')
count_mat.shape

(41580, 1999)

In [ ]:
rawcount_mat = sio.mmread(data_path+'/tenxCount.mtx')
rawcount_mat.shape

(41580, 1999)

In [ ]:
#Center and scale log-normalized data
scaled_mat = scale(count_mat)

In [ ]:
meta = pd.read_csv(data_path+'/metadata.csv',index_col = 0)
meta.head()

,sample_name,cell_barcode,cluster_color,cluster,cluster_id,cell_types,sex_label,batch_indices,cell_counts,n_genes,percent_mito,pass_count_filter,pass_mito_filter
0,10x_VMH_Female_Control_1,4_AAACCTGAGCGCCTCA,#9162FF,Nr5a1_4,24.0,undefined,F,0,1427.0,926,13.594954,True,False
1,10x_VMH_Female_Control_1,4_AAACCTGCACAGTCGC,#96FF2E,Tsix_Esr1_1,3.0,undefined,F,0,4508.0,2310,9.272405,True,False
2,10x_VMH_Female_Control_1,4_AAACCTGCAGATTGCT,#66ABC2,Dlk1_3,16.0,undefined,F,0,3447.0,1995,5.483029,True,False
3,10x_VMH_Female_Control_1,4_AAACCTGCATACTCTT,#96FF2E,Tsix_Esr1_1,3.0,undefined,F,0,7004.0,3286,4.483152,True,True
4,10x_VMH_Female_Control_1,4_AAACCTGGTCCGTTAA,#24EFCD,Scgn,20.0,undefined,F,0,2490.0,1492,9.277108,True,False


In [ ]:
clusters = np.unique(meta['cluster'].values)
map_dict = {}
for i, c in enumerate(clusters):
  map_dict[c] = i
new_labs = [map_dict[c] for c in meta['cluster'].values]

In [ ]:
adata = anndata.AnnData(count_mat, obs = meta)
adata.X = np.nan_to_num(adata.X)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [ ]:
adata2 = anndata.AnnData(rawcount_mat, obs = meta)
adata2.X = np.nan_to_num(adata2.X)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  """Entry point for launching an IPython kernel.


In [ ]:
def knn_infer(embd_space, labeled_idx, labeled_lab, unlabeled_idx,n_neighbors=50):
	"""
	Predicts the labels of unlabeled data in the embedded space with KNN.
	Parameters
	----------
	embd_space : ndarray (n_samples, embedding_dim)
		Each sample is described by the features in the embedded space.
		Contains all samples, both labeled and unlabeled.
	labeled_idx : list
		Indices of the labeled samples (used for training the classifier).
	labeled_lab : ndarray (n_labeled_samples)
		Labels of the labeled samples.
	unlabeled_idx : list
		Indices of the unlabeled samples.
	Returns
	-------
	pred_lab : ndarray (n_unlabeled_samples)
		Inferred labels of the unlabeled samples.
	"""

	# obtain labeled data and unlabled data from indices
	labeled_samp = embd_space[labeled_idx, :]
	unlabeled_samp = embd_space[unlabeled_idx, :]

	from sklearn.neighbors import KNeighborsClassifier

	knn = KNeighborsClassifier(n_neighbors=n_neighbors)
	knn.fit(labeled_samp, labeled_lab)

	pred_lab = knn.predict(unlabeled_samp)
	return pred_lab


### **Prediction Accuracy for Cell Type Labels Across Benchmarks**
Tests accuracy on sex labels as well


Set up metadata

In [ ]:
lab1 = list(meta.cluster)
lab2 = list(meta.sex_label)
# lab3 = list(meta.medical_cond_label)
lab4 = list(meta.cluster)

allLabs = np.array([lab1])
allLabs2 = np.array([lab1,lab2])

nanLabs = np.array([[np.nan]*len(lab1)])

#Shuffled labels for over-fitting check
shuff_lab1 = random.sample(lab1, len(lab1))
shuff_lab2 = random.sample(lab2, len(lab2))
shuff_allLabs = np.array([shuff_lab1,shuff_lab2])

clus_colors = list(pd.unique(meta.cluster_color))

sex_colors = ['#abacb7','#F8C471']

In [ ]:
labdict = {}
count = 0
for p in np.unique(meta.cluster):
  labdict[p] = count
  count += 1

lab4 = [labdict[i] for i in meta.cluster]

First test 2D space predictions (t-SNE, UMAP, UMAP-Supervised)

In [ ]:
ndims = 2
acc_score_2D = []

for i in range(3):
  reducer = umap.UMAP(n_components = ndims)
  tsne = TSNE(n_components = ndims)


  tsvd = TruncatedSVD(n_components=pcs)
  x_pca = tsvd.fit_transform(scaled_mat)

  pcaUMAP = reducer.fit_transform(x_pca)
  pcaTSNE = tsne.fit_transform(x_pca)

  #Partially labeled UMAP

  labels = np.array([lab4]).copy().astype(np.int8)
  train_inds = np.random.choice(len(scaled_mat), size = int(0.7*len(scaled_mat)),replace=False) #0.7 for training fraction
  #Set 30% to no label (nan)
  unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
  labels[:, unlab_inds] = -1

  pcaUMAPLab = reducer.fit_transform(x_pca,y=labels[0])

  preds = knn_infer(pcaUMAPLab, train_inds, adata.obs.cluster_id.values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs.cluster_id.values[unlab_inds], preds)
  acc_score_2D.append(acc)



  preds = knn_infer(pcaUMAP, train_inds, adata.obs.cluster_id.values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs.cluster_id.values[unlab_inds], preds)
  acc_score_2D.append(acc)

  preds = knn_infer(pcaTSNE, train_inds, adata.obs.cluster_id.values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs.cluster_id.values[unlab_inds], preds)
  acc_score_2D.append(acc)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [ ]:
print(acc_score_2D)

[0.7692184368737475, 0.6889779559118236, 0.7565531062124249, 0.7739478957915832, 0.6800801603206413, 0.7609619238476953, 0.7727454909819639, 0.6808016032064128, 0.7596793587174349]


In [ ]:
acc_score_2D = [0.7692184368737475, 0.6889779559118236, 0.7565531062124249, 0.7739478957915832, 0.6800801603206413, 0.7609619238476953, 0.7727454909819639, 0.6808016032064128, 0.7596793587174349]

Get prediction accuracy in PCA 40d --> 2D space

In [ ]:
ndims = 2
acc_score_40_2D = []

for i in range(3):
  reducer = umap.UMAP(n_components = ndims)
  tsne = TSNE(n_components = ndims)


  tsvd = TruncatedSVD(n_components=pcs3)
  x_pca = tsvd.fit_transform(scaled_mat)

  pcaUMAP = reducer.fit_transform(x_pca)
  pcaTSNE = tsne.fit_transform(x_pca)

  #Partially labeled UMAP

  labels = np.array([lab4]).copy().astype(np.int8)
  train_inds = np.random.choice(len(scaled_mat), size = int(0.7*len(scaled_mat)),replace=False) #0.7 for training fraction
  #Set 30% to no label (nan)
  unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
  labels[:, unlab_inds] = -1

  pcaUMAPLab = reducer.fit_transform(x_pca,y=labels[0])

  preds = knn_infer(pcaUMAPLab, train_inds, adata.obs.cluster_id.values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs.cluster_id.values[unlab_inds], preds)
  acc_score_40_2D.append(acc)



  preds = knn_infer(pcaUMAP, train_inds, adata.obs.cluster_id.values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs.cluster_id.values[unlab_inds], preds)
  acc_score_40_2D.append(acc)

  preds = knn_infer(pcaTSNE, train_inds, adata.obs.cluster_id.values[train_inds], unlab_inds)
  acc = accuracy_score(adata.obs.cluster_id.values[unlab_inds], preds)
  acc_score_40_2D.append(acc)

In [ ]:
print(acc_score_40_2D)

PCA 50D

In [ ]:

acc_scorePCA = []
acc_scorePCA2 = []

for i in range(3):

  tsvd = TruncatedSVD(n_components=pcs)
  x_pca = tsvd.fit_transform(scaled_mat)

  labels = np.array([lab1])
  train_inds = np.random.choice(len(scaled_mat), size = int(0.7*len(scaled_mat)),replace=False)
  unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
  labels[:, unlab_inds] = np.nan

  unlabeled_idx = []
  for i in range(len(adata)):
      if i not in train_inds:
          unlabeled_idx.append(i)

  preds = knn_infer(x_pca, train_inds, adata.obs.cluster.values[train_inds], unlabeled_idx)
  acc = accuracy_score(adata.obs.cluster.values[unlabeled_idx], preds)
  acc_scorePCA.append(acc)

  preds2 = knn_infer(x_pca, train_inds, adata.obs.sex_label.values[train_inds], unlabeled_idx)
  acc2 = accuracy_score(adata.obs.sex_label.values[unlabeled_idx], preds2)
  acc_scorePCA2.append(acc2)
  # print(f"nnNCA fit in {toc - tic:0.4f} seconds")

In [ ]:
print(acc_scorePCA)
print(acc_scorePCA2)

[0.8142685370741483, 0.8189178356713427, 0.8188376753507014]
[0.9039679358717435, 0.9084569138276554, 0.906372745490982]


In [ ]:
acc_scorePCA = [0.8142685370741483, 0.8189178356713427, 0.8188376753507014]
acc_scorePCA2 = [0.9039679358717435, 0.9084569138276554, 0.906372745490982]

PCA 40D

In [ ]:

acc_scorePCA40D = []

for i in range(3):

  tsvd = TruncatedSVD(n_components=pcs3)
  x_pca = tsvd.fit_transform(scaled_mat)

  labels = np.array([lab1])
  train_inds = np.random.choice(len(scaled_mat), size = int(0.7*len(scaled_mat)),replace=False)
  unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
  labels[:, unlab_inds] = np.nan

  unlabeled_idx = []
  for i in range(len(adata)):
      if i not in train_inds:
          unlabeled_idx.append(i)

  preds = knn_infer(x_pca, train_inds, adata.obs.cluster.values[train_inds], unlabeled_idx)
  acc = accuracy_score(adata.obs.cluster.values[unlabeled_idx], preds)
  acc_scorePCA40D.append(acc)


  # print(f"nnNCA fit in {toc - tic:0.4f} seconds")

In [ ]:
print(acc_scorePCA40D)


PCA 100D

In [ ]:

acc_scorePCA100D = []

for i in range(3):

  tsvd = TruncatedSVD(n_components=100)
  x_pca = tsvd.fit_transform(scaled_mat)

  labels = np.array([lab1])
  train_inds = np.random.choice(len(scaled_mat), size = int(0.7*len(scaled_mat)),replace=False)
  unlab_inds = [i for i in range(len(adata)) if i not in train_inds]
  labels[:, unlab_inds] = np.nan

  unlabeled_idx = []
  for i in range(len(adata)):
      if i not in train_inds:
          unlabeled_idx.append(i)

  preds = knn_infer(x_pca, train_inds, adata.obs.cluster.values[train_inds], unlabeled_idx)
  acc = accuracy_score(adata.obs.cluster.values[unlabeled_idx], preds)
  acc_scorePCA100D.append(acc)


  # print(f"nnNCA fit in {toc - tic:0.4f} seconds")

In [ ]:
print(acc_scorePCA100D)

[0.8074549098196393, 0.8089779559118236, 0.8082565130260521]


In [ ]:
acc_scorePCA100D = [0.8074549098196393, 0.8089779559118236, 0.8082565130260521]

### **Save Analysis Output**

In [ ]:
vals = pd.DataFrame()

vals['Accuracy'] = acc_scorePCA +acc_scorePCA2 + acc_score_2D  +acc_scorePCA100D + acc_score_40_2D + acc_scorePCA40D#+ netAE_score + netAE_score2

In [ ]:
vals['Embed'] = ['PCA 50D']*3 + ['PCA 50D']*3 + ['PCA 50D UMAP Sup.','PCA 50D UMAP','PCA 50D t-SNE']*3 + ['PCA 100D']*3+['PCA 40D UMAP Sup.','PCA 40D UMAP','PCA 40D t-SNE']*3 + ['PCA 40D']*3 #+ ['netAE']*2

In [ ]:
vals['Label'] = ['CellType1']*3 + ['Gender2']*3 + ['CellType1']*12 + ['CellType1']*12 #+  ['CellType1'] #+  ['Gender2']

In [ ]:
vals


,Accuracy,Embed,Label
0,0.794132,LDVAE,CellType1
1,0.791727,LDVAE,CellType1
2,0.791406,LDVAE,CellType1
3,0.791727,SCANVI,CellType1
4,0.797739,SCANVI,CellType1
5,0.795815,SCANVI,CellType1
6,0.830621,Recon MCML,CellType1
7,0.832625,Recon MCML,CellType1
8,0.827335,Recon MCML,CellType1
9,0.726814,NCA 100% MCML,CellType1


In [ ]:
from google.colab import files

vals.to_csv('all10XPreds_22.csv')
files.download('all10XPreds_22.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>